In [15]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import linspace, polyval, polyfit, sqrt, stats, randn

# body, leftUpperArm, RightUpperArm, RightFoot, LeftFoot, LeftHand, RightHand
bodyPartsGroups= [0,1,2,3,4,5,6,7]
# constants 
path = "E:\\Andreea\\Master\\Thesis\\Git\\TheOne\\SmartuistWrapper\\tapResultParticipants.csv"
pathH = "C:\\Users\\huber\\OneDrive\\Dokumenty\\thesis-hubert\\unity\\SmartuistWrapper\\tapResultParticipants.csv"
playerNr = 46
players = [2, 7, 11, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62] #26, 58
males = [0, 4, 5 , 6, 7, 12, 14, 15]
females = [1, 2, 3, 8, 9, 10, 11, 13]
gender = [0, 1, 1, 1, 0, 0, 0,  0,  1, 1, 1, 1, 0, 1, 0, 0]
armLength = np.array([28, 23.5, 27, 27, 28, 27, 26, 28,  23, 26, 24, 26, 27, 24, 29, 29])
height = np.array([1.7, 1.73, 1.65, 1.72, 1.77, 1.82, 181, 1.79, 1.6, 1.7, 1.7, 1.65, 1.8, 1.78, 184, 1.85 ])
idxLength = 28
idxGain = 29
blue = 'royalblue'
red = 'crimson'
col1 = 'darkmagenta'
col2 = 'goldenrod'
col3 = 'mediumturquoise'
col4 ='forestgreen'
col5 = 'navy'
plt.rcParams['figure.figsize'] = [20, 5]
cols3 = [col3,blue, col5]

playersBaseline = [4, 9, 13, 16, 19, 24, 28, 31, 36, 39, 43, 48, 51, 56, 60, 63]
playersOptimal  = [3, 8, 12, 15, 20, 23, 27, 32, 35, 40, 44, 47, 52, 55, 59, 64]

#GRAPHS#
SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [26]:
def readFromFile(path):
    results = [0] * 100
    f = open(path, "r")
    player = -1
    for x in f:
        line = x.split(',')
        if len(line) == 31:
            if int(line[0]) == player:
                results[player].append(line)
            else:
                player = int(line[0])
                results[player] = []
                results[player].append(line)
    f.close()
    return results


# a list with the results at idx for the given player    
def getResultPerPlayer(p, results, idx, unique ):
    tmp = results[p]
    values = []
    for words in tmp:
        values.append(float(words[idx]))
    if (unique == True):
        values =  [x for i, x in enumerate(values) if i == values.index(x)]
    return values

# sort list1 - use the indexes of the sorted list1 to rearrage list2    
def getCombinedPlot(list1, list2):
    idx   = np.argsort(list1)
    list1 = np.array(list1)[idx]
    list2 = np.array(list2)[idx]
    return (list1, list2)

def sortBasedOn1(list1, list2):
    idx   = np.argsort(list1)
    list1 = np.array(list1)[idx]
    list2 = np.array(list2)[idx]
    return (list1, list2)

# get avg effort and distance per body parts
def EffortPerBodyPartPerPlayer(p, results, batchSize):
    effortBodyParts = [ 0 for i in range(7)]
    distBodyParts = [ 0 for i in range(7)]
    tmp = results[p]
    ids = []
    time = []
    counter = 0
    
    for words in tmp:            
        for i in range (7):
            effortBodyParts[i] += float(words[21+i])
            distBodyParts[i] += float(words[14+i]) 
        time.append(float(words[10]))
        ids.append(float(words[11]))
        counter = counter + 1

    for i in range(7):
        distBodyParts[i] /= counter
    return (effortBodyParts, distBodyParts, time, ids)

def getResults(results, players):
    efforBodyParts = []
    distBodyParts = []
    time = []
    ids = []
    for p in players:
        tmp = results[p]
        effort, dist, t, i= EffortPerBodyPartPerPlayer(p, results, 10)
        efforBodyParts.append(effort)
        distBodyParts.append(dist)
        time.append(t)
        ids.append(i)
    return ( efforBodyParts, distBodyParts, time, ids)




In [27]:
results = readFromFile(path)
efforBodyParts, distBodyParts, times, ids = getResults(results, players)

In [28]:
baseDist = [[] for i in range(7)]
optDist = [[] for i in range(7)]



for (b,o) in list(zip(playersBaseline, playersOptimal)):
    (effortsb, dist_b,tb,idb) = EffortPerBodyPartPerPlayer(b,results,10)
    (effortso, dist_o, to, ido) = EffortPerBodyPartPerPlayer(o,results,10)
    for i in range(7):
        baseDist[i].append(dist_b[i])
        optDist[i].append(dist_o[i])


# they are kind of uniformly distributed -> we can use parametric tests
for i in range(7):
    t,p = stats.wilcoxon(baseDist[i], optDist[i])
    for data in [baseDist[i], optDist[i]]:
        print(data)
        median = np.median(data)
        upper_quartile = np.percentile(data, 75)
        lower_quartile = np.percentile(data, 25)
        iqr = upper_quartile - lower_quartile
        print("{:.2f} & {:.2f}".format(median, iqr))
        
   # print("{:.3f} \\\\".format(p))
    print("{:.3f} & {:.3f} \\\\".format(t,p))
#     normal = stats.normaltest(baseDist[i])
#     print("Normal | Base: ", normal)
#     normal = stats.normaltest(optDist[i])
#     print("Normal | Opti: ", normal)

[1.144540121333333, 1.0302043916666666, 1.2114535238095239, 1.30153865, 1.2429712049999997, 1.1405177085, 1.2949956460000003, 1.1416344325, 1.2986075679999998, 1.0349129940000001, 1.3129102099999996, 1.165109583, 1.2124860159999997, 1.2332522785000002, 1.340109045, 1.3925087750000003]
1.22 & 0.16
[0.5075824276666667, 0.585155965, 0.8788557335, 0.7673411900000002, 0.6239296835, 0.48289630000000006, 0.6879366455, 0.5709414225, 0.388118935, 0.7068687115000001, 0.6300392505000001, 0.4953707604999999, 0.529000138, 0.8425155180000001, 0.5778484899999999, 0.9798130879999999]
0.60 & 0.20
0.000 & 0.000 \\
[1.22642435, 1.0198964233333334, 1.3314212904761906, 1.2813069049999999, 1.3372503050000002, 1.2238158085000002, 1.32799037, 1.2581619700000002, 1.4009135249999998, 1.1906361204999998, 1.247425685, 1.242280005, 1.375950985, 1.276982991, 1.3729591099999998, 1.4953352299999998]
1.28 & 0.11
[0.54519672, 0.5703155593333332, 0.9637525364999998, 0.74662187, 0.7467452300000001, 0.553842315, 0.7330289